In [1]:
import sys
import os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist( normalize=True, one_hot_label=True)
print (x_train.shape)
print (t_train.shape)
t_train

(60000, 784)
(60000, 10)


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [2]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
print (batch_mask)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

[28328 25967 48266 17073 18935 14963 33050 13437 48177 42112]


In [3]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [4]:
y = np.random.rand(5, 10)
print(y.shape)
print(y)
t = [2, 7, 0, 9, 4]
y[np.arange(5), t]

(5, 10)
[[0.82826773 0.28565807 0.65961315 0.90705437 0.12837826 0.76557841
  0.68807382 0.65255403 0.33238863 0.41522294]
 [0.17793808 0.1824665  0.62499632 0.03661908 0.29943551 0.8970281
  0.02849569 0.14720037 0.24678642 0.07233532]
 [0.34319732 0.96277878 0.65608706 0.30479798 0.98603098 0.22041677
  0.75021963 0.79164744 0.2736975  0.25066214]
 [0.52143063 0.50840368 0.51527896 0.68424829 0.71766439 0.89722024
  0.83649973 0.82260773 0.74732643 0.50847195]
 [0.01472852 0.21010571 0.71665597 0.94128479 0.99402156 0.65663555
  0.63165957 0.30669646 0.80778747 0.92697605]]


array([0.65961315, 0.14720037, 0.34319732, 0.50847195, 0.99402156])

In [5]:
import sys
import os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)

    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])

net = simpleNet()

In [6]:
net = simpleNet()
print(net.W)

[[ 1.28092469 -0.54957952 -0.196798  ]
 [-0.12743124 -2.04133797 -0.25069018]]


In [7]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)


[ 0.6538667  -2.16695189 -0.34369997]


In [8]:
np.argmax(p)

np.int64(0)

In [9]:
t = np.array([0, 0, 1])
net.loss(x, t)


np.float64(1.3540742193898603)

In [10]:
x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])
def f(w):
    return net.loss(x, t)
dW = numerical_gradient(f, net.W)

print(dW)

[[ 0.42006998  0.02501818 -0.44508816]
 [ 0.63010498  0.03752727 -0.66763224]]


In [11]:
import sys
import os
sys.path.append(os.pardir)
import numpy as np
from common.functions import sigmoid, sigmoid_grad, softmax
from common.gradient import numerical_gradient

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init=0.01):
        self.params = {}
        self.params['W1'] = weight_init * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    

    def numerical_gradient(self, x, t):
        def loss_W(W):
            return self.loss(x, t)

        grads = {}

        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        dz1 = np.dot(dy, W2.T)
        da1 = sigmoid_grad(a1) * dz1
        grads['W1'] = np.dot(x.T, da1)
        grads['b1'] = np.sum(da1, axis=0)

        return grads

In [12]:
import sys
import os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist


(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'